# dimensions
We can extract a few features from the frames' image size. We can also designate several "points of interest" in the frame, which would be useful, for example, in detecting if a character's face is positioned in the frame according to the rule-of-thirds.

In [1]:
import cv2
import os

In [2]:
film = 'teen_spirit'
frame = 1335
frame_folder = os.path.join('../frame_per_second', film)
img_path = frame_folder + '/' + film + '_frame' + str(frame) + '.jpg'
image = cv2.imread(img_path)

# Basic Dimensions
After loading the image, we can get the height and weight. From there, we can calculate the aspect ratio. In this case, it's 2.4, a very common cinema widescreen ratio.

In [3]:
height = image.shape[0]
height

356

In [4]:
width = image.shape[1]
width

854

In [5]:
aspect_ratio = round(width/height, 2)
aspect_ratio

2.4

# Artificial Aspect Ratio
Films sometimes switch aspect ratios for various reasons. Two common purposes:
1. The characters are watching a film-within-a-film. The aspect ratio will switch to something more cinematic by blocking off the top and bottom of the image with black bars.
2. The audience is watching footage from the past. This is apparent in the opening credits of *Succession*, where "vintage", sepia-toned footage of the main characters as children is interspersed with modern-day footage. The modern-day footage is in the show's natural aspect ratio, while vintage footage is in a narrower aspect ratio, with black bars on the left and right.
To detect an artificial aspect ratio, we can search for black bars in either scenario: top/bottom, or left/right.

### Black Bars on Top and Bottom - Cinematic Black Rows
We'll first detect cinematic black bars on the top and bottom of the frame. In this example from *Rides Start at 10:00*, black rows are used in a film-within-a-film context.

In [6]:
film = 'rides'
frame = 700
frame_folder = os.path.join('../frame_per_second', film)
img_path = frame_folder + '/' + film + '_frame' + str(frame) + '.jpg'
image = cv2.imread(img_path)

We first define a function that will help detect rows with all-black pixels. Because of video compression, some video files may not achieve "true" black in these situations, and we'll set a tolerance of 5 (on the RGB 0-255) scale.

In [7]:
def black_row(image, row_selection):
    
    total_pixels = image.shape[1]
    black_pixels = 0

    for pixel in image[row_selection]:
        if pixel.mean() < 5:
            black_pixels += 1
    
    if black_pixels == total_pixels:
        return True
    else:
        return False

In [8]:
black_row(image, 3) #all-black pixels detected on the fourth-from-top row

True

Next, we can iteratively search from the top/bottom edges, and approach the center until we don't find any more all-black rows. We're left with the highest and lowest rows that are part of the black bars. We can define the "true height" by subtracting the two - the true height is comprised of the remaining rows once we remove the black bars.

In [9]:
def true_height(image):
    top_row = 0
    bottom_row = image.shape[0] - 1
    search_flag = True
    first_flag = True

    while search_flag == True:
        if black_row(image, top_row) and black_row(image, bottom_row):
            top_row += 1
            bottom_row -= 1
            first_flag = False
        elif first_flag == True:
            return image.shape[0]   # if the first search doesn't yield black columns, simply return frame height
        else:
            search_flag = False

        bottom_row += 1      # necessary because we had to subtract 1 when declaring bottom_row
            
    return bottom_row - top_row

In [10]:
print(image.shape[0])
print(true_height(image))

479
418


### Black Bars on Left and Right - Vintage Black Columns
Next, we'll look detect black bars on the left and right of the frame. In this example from *Vault*, black columns are used in a vintage context to show footage from the '70s.

In [11]:
film = 'vault'
frame = 205
frame_folder = os.path.join('../frame_per_second', film)
img_path = frame_folder + '/' + film + '_frame' + str(frame) + '.jpg'
image = cv2.imread(img_path)

Again, we first define a function to seek a single entity of the black bar: an all-black column.

In [12]:
def black_column(image, col_selection):
    
    total_pixels = image.shape[0]
    black_pixels = 0

    for row in image:
        if row[col_selection].mean() < 5:
            black_pixels += 1
    
    if black_pixels == total_pixels:
        return True
    else:
        return False

With this function, we start from the left- and right-most edge columns, and work our way inwards until we get to columns with actual content. The "true width" is much smaller than the actual frame file's width.

In [13]:
def true_width(image):
    left_column = 0
    right_column = image.shape[1] - 1    # list starts at 0, while .shape starts at 1
    search_flag = True
    first_flag = True

    while search_flag == True:
        if black_column(image, left_column) and black_column(image, right_column):
            left_column += 1
            right_column -= 1
            first_flag = False
        elif first_flag == True:     # if the first search doesn't yield black columns, simply return frame width
            return image.shape[1]
        else:
            search_flag = False

        right_column += 1      # necessary because we had to subtract 1 when declaring right_column
            
    return right_column - left_column

In [14]:
print(image.shape[1])
print(true_width(image))

854
694


### True Aspect Ratio
We can use these "true" widths and heights to find the actual aspect ratio, and compare it to the original aspect ratio (of the frame image size). If there's a difference, this part of the frame may have special qualities (vintage, film-within-a-film, etc.)

In [15]:
print('Frame height:', image.shape[0])
print('True image height:', true_height(image))
print('Frame width:', image.shape[1])
print('True image width:', true_width(image))

Frame height: 362
True image height: 362
Frame width: 854
True image width: 694


In [16]:
print('Frame aspect ratio:', round(image.shape[1]/image.shape[0], 2))
print('True aspect ratio:', round(true_width(image)/true_height(image), 2))

Frame aspect ratio: 2.36
True aspect ratio: 1.92


# Points of Interest
We can designate specific points in the frame as points of interest. 
### Center
Certain directors, most notably Wes Anderson, like to compose characters, objects, or other physical elements in the center of the frame. In addition, some "split-screen" scenes can be identified by a central border and left- and right-half images.

In [17]:
half_height = round(height * (1/2))
half_height

178

In [18]:
half_width = round(width * (1/2))
half_width

427

In [19]:
center_point = (half_height, half_width)
center_point

(178, 427)

In [20]:
type(center_point)

tuple

### Rule of Thirds
The rule of thirds is an important axiom of cinemtography and photography. Rather than placing characters, objects, or horizon lines in the dead-center of frame (one-half), they can be placed at one- or two-thirds of the total width or height. Horizon lines are often placed at one-third of frame height, allowing two-thirds of the frame to be occupied by sky.

In [21]:
one_third_height = round(height * (1/3))
two_thirds_height = round(height * (2/3))
one_third_width = round(width * (1/3))
two_thirds_width = round(width * (2/3))
print('1/3 height:', one_third_height)
print('2/3 height:', two_thirds_height)
print('1/3 width:', one_third_width)
print('2/3 width:', two_thirds_width)

1/3 height: 119
2/3 height: 237
1/3 width: 285
2/3 width: 569


Cinematographers will often position characters or other prominent objects at the intersection of (1/3) or (2/3) width and height. There are four points where the rule of thirds applies.

In [22]:
thirds_point_a = (one_third_height, one_third_width)
thirds_point_b = (two_thirds_height, one_third_width)
thirds_point_c = (one_third_height, two_thirds_width)
thirds_point_d = (two_thirds_height, two_thirds_width)
print('Rule of Thirds Point A:', thirds_point_a)
print('Rule of Thirds Point B:', thirds_point_b)
print('Rule of Thirds Point C:', thirds_point_c)
print('Rule of Thirds Point D:', thirds_point_d)

Rule of Thirds Point A: (119, 285)
Rule of Thirds Point B: (237, 285)
Rule of Thirds Point C: (119, 569)
Rule of Thirds Point D: (237, 569)
